## Multilayer Neural Network

## Key Points

* Perceptrons never converge on non-linearly separable data
* Logistic Regresson will converge but only for two classes
* For multiple classes Softmax regression
* Softmax regression only creates linear boundaries
* Multilayer perceptron can handle multiple classes and complex boundaries but needs more data and tuning
* Multilayer perceptron is referred to as a fully-connected feedforward neural network
* Multilayer is essentially softmax with some extra hidden layers

### Some best practices

* Start with Logistic Regression and/or Softmax to get a baseline, then try to beat that performance with a more complicated model. 
* For the output layer we use the softmax activation function and 2 nodes for binary classification
* For the hidden layers we could use the sigmoid activation function however it is now more commone to use the ReLU function

##### ReLU function

* Rectified Linear Unit
* Piecwise sigma(z) = z if z > 0, else 0, or sigma(z) = max(0,z)
* most common


##### Multilayer perceptron basic arhictecture design considerations

* Hidden Layer must have non-linear activation function
* Deep vs. Wide?
* Wide and Shallow: ex. 1 hidden layer with many nodes
* Narrow and deep: ex. many hidden layers each with a few nodes
* Both can technically work but the have pros and cons
* Wide and Shallow: Needs lots of hidden units and is prone to memorization (vs learning)
* Narrow and Deep: Needs less hidden units, generalized better, however it is harder to train due to vanishing/exploding gradient problems
* How do we initialize the weights? 
    * If we initialize all to 0 then all hidden units have the same value, which makes them redundant
    * Initialize to small random numbers

## Bulding the multilayer perceptron

In [1]:
%load_ext watermark
%watermark -v -p numpy,pandas,matplotlib,torch --conda

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.14.0

numpy     : 1.24.3
pandas    : 2.0.2
matplotlib: 3.7.1
torch     : 2.0.1

conda environment: n/a



In [2]:
#### Loading the Dataset